In [ ]:
!pip install pycoingecko
!pip install configparser
!pip install tabulate

## Source : https://github.com/man-c/pycoingecko

CoinGecko API wrapper
PyPi Version GitHub

In [1]:
import time 
import datetime
import pandas as pd
import requests
import configparser
from pycoingecko import CoinGeckoAPI

cg = CoinGeckoAPI()

In [2]:
config = configparser.ConfigParser()
config.read('..\..\config.conf')
config.sections()
config["telegram_RedNavi_bot"]['bot_token']

'247419374:AAGMXRpFeh26gzp4vF-I7VK2M4EEpNONCmU'

In [3]:
coins =['bitcoin', 'bitcoin-cash', 'ethereum', 'litecoin', 'ripple']
currency="MYR"
coin_market=dict()
coin_market_hourly=dict()
for coin in coins:
    print (coin)
    coin_market[coin]=(cg.get_coin_market_chart_by_id(id=coin,vs_currency="MYR",days="90",interval="daily"))
    coin_market_hourly[coin]=(cg.get_coin_market_chart_by_id(id=coin,vs_currency="MYR",days="90",interval="hourly"))

bitcoin
bitcoin-cash
ethereum
litecoin
ripple


In [16]:
def convert_unix(time): #For fuck sake this is annoying To convert time 
    #return datetime.datetime.fromtimestamp(int(str(time)[:10])).strftime('%Y-%m-%d %H:%M:%S')
    return datetime.datetime.fromtimestamp(int(str(time)[:10])).strftime('%d/%m/%y %a %H:%M')
    #return datetime.datetime.fromtimestamp(int(str(time)[:10])).strftime('%Y-%m-%d')

def convert_data_frame(dict_data):
    df =pd.DataFrame(dict_data["prices"])
    df.columns = ['posix', 'price']
    #df.date=df.apply(lambda row : add(row['A'],row['B'], row['C']), axis = 1)vconvert_unix(time)
    df["date"]=df.apply(lambda row : convert_unix(row['posix']), axis = 1)
    price_now=df['price'].values[len(df.price)-1]
    df["diff_between_now"]=(price_now/df['price']-1)*100
    df["change"]= df.price.pct_change() * 100
    
    return df[["date","price","diff_between_now","change"]].round(2)
print(convert_data_frame(coin_market['bitcoin']))


def telegram_bot_sendtext(bot_message):
    bot_token = config["telegram_RedNavi_bot"]['bot_token']
    #bot_chatID = '269721616'562593592
    bot_chatID = config["telegram_RedNavi_bot"]['bot_chatID']
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message
    response = requests.get(send_text)
    return response.json()

                  date      price  diff_between_now  change
0   14/03/21 Sun 08:00  252495.99            -39.42     NaN
1   15/03/21 Mon 08:00  244728.49            -37.50   -3.08
2   16/03/21 Tue 08:00  229499.42            -33.35   -6.22
3   17/03/21 Wed 08:00  233799.66            -34.57    1.87
4   18/03/21 Thu 08:00  243230.02            -37.11    4.03
..                 ...        ...               ...     ...
86  08/06/21 Tue 08:00  138087.31             10.78   -6.65
87  09/06/21 Wed 08:00  138255.46             10.64    0.12
88  10/06/21 Thu 08:00  154627.13             -1.07   11.84
89  11/06/21 Fri 08:00  152060.07              0.60   -1.66
90  11/06/21 Fri 16:22  152967.23              0.00    0.60

[91 rows x 4 columns]


In [119]:
coin_market_hourly=dict()
current_price=cg.get_price(ids=coins, vs_currencies=currency)
message="Current coin price in MYR is :\n"
for i in current_price:
    message = message+(i+": MYR "+str(current_price[i]['myr'])+"\n")
print(message)
test = telegram_bot_sendtext(message)

Current coin price in MYR is :
bitcoin: MYR 151391
ethereum: MYR 10153.63
ripple: MYR 3.57
litecoin: MYR 702.75
bitcoin-cash: MYR 2460.65



In [48]:
test = telegram_bot_sendtext("Testing Telegram bot")
print(test)


KeyError: 'chat_id'

In [48]:
convert_data_frame(coin_market_hourly['bitcoin']).tail()
print(str (convert_data_frame(coin_market_hourly['bitcoin']).tail()[["date","price","diff_between_now"]].to_csv(index=False,sep='|')))
#test = telegram_bot_sendtext((str (convert_data_frame(coin_market_hourly['bitcoin'][[da]]).tail().to_csv(index=False,sep='|'))))

date|price|diff_between_now
21-06-11 08:23|152620.06084973205|-0.7685842137980181
21-06-11 09:17|149745.64957094996|1.1361916615787893
21-06-11 10:07|149717.64615999567|1.1551083251123861
21-06-11 11:09|150061.27526873798|0.9234706847135454
21-06-11 11:31|151447.04715495207|0.0



In [36]:
test_string="""<pre language="c++">code</pre>,"""
test = telegram_bot_sendtext(test_string)

In [120]:
from tabulate import tabulate
df=convert_data_frame(coin_market_hourly['bitcoin']).tail()[["date","price","diff_between_now","diff_previous_day"]]
#print(tabulate(df, headers='keys', tablefmt='psql'))
#print(tabulate(df, headers='keys', tablefmt='fancy_grid'))
print(tabulate(df.values.tolist(), headers=['date',"price","Diff_between","Diff_prev"], tablefmt='psql'))
print(df.values.tolist())
telegram_bot_sendtext(tabulate(df.values.tolist(), headers=['date',"price","Diff_between","Diff_prev"], tablefmt='psql'))

#telegram_bot_sendtext("""<code>inline fixed-width code</code>""")

KeyError: 'bitcoin'

In [17]:
convert_data_frame(coin_market_hourly['bitcoin'])
df_hourly=convert_data_frame(coin_market_hourly['bitcoin'])[["date","change"]]
df_hourly.tail(5)

,date,change
2158,11/06/21 Fri 13:11,1.59
2159,11/06/21 Fri 14:16,-1.32
2160,11/06/21 Fri 15:07,-0.52
2161,11/06/21 Fri 16:02,1.28
2162,11/06/21 Fri 16:29,0.07
